# Exploring CarMax trade-ins

## Import libraries and data

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [63]:
data = pd.read_csv("../data/viz.csv").drop("Unnamed: 0", axis = 1)

## First, let's take a quick look at the data to remind ourselves what it looks like.

In [5]:
data.head()

,price,appraisal_offer,make_appraisal,model_appraisal,trim_level_premium_appraisal,model_year_appraisal,mileage_appraisal,engine_appraisal,mpg_city_appraisal,mpg_highway_appraisal,...,mid_size_appraisal,pickup_appraisal,small_suv_appraisal,sports_car_appraisal,van_appraisal,cylinders,cylinders_appraisal,region,color_grouped,color_grouped_appraisal
0,24000,9000,Ford,Escape,True,2012,39300,1.6,22.0,29,...,False,False,True,False,False,6,4,Midwest,Black,White
1,33000,14600,Toyota,Tacoma,True,2014,105800,3.5,19.0,24,...,False,True,False,False,False,6,6,West,Black,Gray
2,25500,3400,Chevrolet,Cruze,False,2008,97300,1.4,28.0,42,...,False,False,False,False,False,4,4,Midwest,Gray,White
3,18700,1100,Chevrolet,Impala,True,2006,145600,3.9,17.0,27,...,False,False,False,False,False,6,6,South,Black,White
4,19500,15000,GMC,Yukon,False,2009,51600,5.3,15.0,21,...,False,False,False,False,False,4,8,West,Red,Black


In [6]:
print(f"Rows: {data.shape[0]}")
print(f"Columns: {data.shape[1]}")

Rows: 111543
Columns: 58


In [7]:
print("Columns:")
data.columns

Columns:


Index(['price', 'appraisal_offer', 'make_appraisal', 'model_appraisal',
       'trim_level_premium_appraisal', 'model_year_appraisal',
       'mileage_appraisal', 'engine_appraisal', 'mpg_city_appraisal',
       'mpg_highway_appraisal', 'horsepower_appraisal',
       'fuel_capacity_appraisal', 'vehicle_type_appraisal', 'color_appraisal',
       'make', 'model', 'trim_level_premium', 'model_year', 'mileage',
       'engine', 'mpg_city', 'mpg_highway', 'horsepower', 'fuel_capacity',
       'vehicle_type', 'color', 'online_appraisal_flag', 'state',
       'days_since_offer', 'cylinders_even', 'cylinders_even_appraisal',
       'cylinders_high', 'cylinders_high_appraisal', 'compact', 'full_size',
       'large_suv', 'luxury', 'medium_suv', 'mid_size', 'pickup', 'small_suv',
       'sports_car', 'van', 'compact_appraisal', 'full_size_appraisal',
       'large_suv_appraisal', 'luxury_appraisal', 'medium_suv_appraisal',
       'mid_size_appraisal', 'pickup_appraisal', 'small_suv_appraisal',
 

In [8]:
data.dtypes.head()

price                            int64
appraisal_offer                  int64
make_appraisal                  object
model_appraisal                 object
trim_level_premium_appraisal      bool
dtype: object

In [9]:
dummy_counter = 0
float_counter = 0
int_counter = 0
obj_counter = 0

for i in data.columns:
    if data[i].dtype == "bool":
        dummy_counter = dummy_counter + 1

    elif data[i].dtype == "float64":
        float_counter = float_counter + 1
        
    elif data[i].dtype == "int64":
        int_counter = int_counter + 1
    
    else:
        obj_counter = obj_counter + 1
        
print(f"Dummy Columns: {dummy_counter}")
print(f"Float Columns: {float_counter}")
print(f"Integer Columns: {int_counter}")
print(f"Object Columns: {obj_counter}")
print(f"Total: {dummy_counter + float_counter + int_counter + obj_counter}")

Dummy Columns: 23
Float Columns: 6
Integer Columns: 17
Object Columns: 12
Total: 58


In [10]:
data.describe()

,price,appraisal_offer,model_year_appraisal,mileage_appraisal,engine_appraisal,mpg_city_appraisal,mpg_highway_appraisal,horsepower_appraisal,fuel_capacity_appraisal,model_year,...,mpg_highway,horsepower,fuel_capacity,days_since_offer,cylinders_even,cylinders_even_appraisal,cylinders_high,cylinders_high_appraisal,cylinders,cylinders_appraisal
count,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,...,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000,111543.000000
mean,21703.676609,8702.079915,2010.131752,80629.963333,2.786724,22.255453,30.227016,214.677882,17.387501,2014.235021,...,29.573653,235.593861,17.456147,1.263011,0.995383,0.991582,0.386721,0.369033,4.946066,4.907964
std,6865.891718,7157.055694,3.887661,43751.178018,1.066918,5.306000,6.181651,73.567761,4.454514,1.878096,...,5.833848,75.972278,4.320093,2.291600,0.067792,0.091365,0.487001,0.482545,1.312961,1.286500
min,3700.000000,0.000000,1992.000000,0.000000,1.000000,10.000000,13.000000,70.000000,7.000000,2005.000000,...,16.000000,70.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000
25%,16500.000000,3000.000000,2008.000000,47100.000000,2.000000,18.000000,25.000000,160.000000,14.000000,2013.000000,...,25.000000,176.000000,14.000000,0.000000,1.000000,1.000000,0.000000,0.000000,4.000000,4.000000
50%,20200.000000,6800.000000,2011.000000,75100.000000,2.500000,22.000000,30.000000,184.000000,17.000000,2014.000000,...,29.000000,206.000000,16.000000,0.000000,1.000000,1.000000,0.000000,0.000000,4.000000,4.000000
75%,25500.000000,12800.000000,2013.000000,106500.000000,3.500000,26.000000,35.000000,278.000000,20.000000,2015.000000,...,33.000000,290.000000,19.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,6.000000
max,95200.000000,69800.000000,2019.000000,482500.000000,7.000000,60.000000,54.000000,702.000000,38.000000,2019.000000,...,59.000000,760.000000,38.000000,7.000000,1.000000,1.000000,1.000000,1.000000,16.000000,8.000000


## Now, let's begin to explore the data by answering some questions we may have.

### How many observations are there per model and make of vehicle appraised?

In [11]:
data["model_appraisal"].value_counts().sort_values(ascending=False).head(10)

model_appraisal
Civic      4695
Altima     4267
Accord     4167
Camry      3633
Escape     3062
F150       3021
Rogue      3018
Elantra    2938
Corolla    2912
Sentra     2504
Name: count, dtype: int64

In [12]:
data["make_appraisal"].value_counts().sort_values(ascending=False).head(10)

make_appraisal
Ford          15822
Nissan        14483
Chevrolet     13365
Toyota        12533
Honda         11673
Jeep           9943
Hyundai        7649
Dodge          5264
Kia            4924
Volkswagen     2716
Name: count, dtype: int64

It looks like Fords are the most observed traded-in brand of car but the Honda Civic is the most commonly traded-in model of car.

### What are the most commonly bought makes and models?

In [19]:
data["model"].value_counts().sort_values(ascending=False).head(10)

model
Civic             3840
Grand Cherokee    3552
Rogue             3285
Camry             3157
Accord            3056
1500              2953
RAV4              2850
Equinox           2746
CR-V              2666
F150              2504
Name: count, dtype: int64

In [20]:
data["make"].value_counts().sort_values(ascending=False).head(10)

make
Toyota        16137
Honda         13437
Jeep          10817
Ford          10660
Chevrolet      9886
Nissan         9452
Hyundai        5436
Kia            4057
Subaru         3764
Volkswagen     3579
Name: count, dtype: int64

Civics are the most popular type of car to purchase after a trade-in but Toyotas are the more popular brand overall. 

### How popular are different colors?

In [22]:
data["color_grouped"].value_counts().sort_values(ascending=False)

color_grouped
White     28353
Black     26463
Gray      24793
Silver    12586
Blue       9904
Red        6259
Other      3185
Name: count, dtype: int64

White is the most popular color car for buyers.

### Can we learn anything about the value of different colors for trade-ins and purchases?

In [27]:
data[["color_grouped_appraisal", "appraisal_offer"]].groupby("color_grouped_appraisal").mean().sort_values(by = "appraisal_offer", ascending=False)

,appraisal_offer
color_grouped_appraisal,
White,9820.125491
Black,9427.602541
Gray,9278.975876
Red,8190.619841
Blue,7454.586694
Silver,6824.636852
Other,6634.317572


In [28]:
data[["color_grouped", "price"]].groupby("color_grouped").mean().sort_values(by = "price", ascending=False)

,price
color_grouped,
Black,22501.753392
White,22109.935457
Gray,21708.155528
Other,21644.992151
Blue,20880.492730
Red,20743.745007
Silver,20241.633561


Even though white sells more and trades in for a higher value on average, black cars appear to command higher prices in the market.

### How are the variables correlated with how much a customer will eventually pay for a car from CarMax?

In [13]:
num_cols = list()
for i in data.columns:
    if data[i].dtype == "int64" or data[i].dtype == "int32" or data[i].dtype == "float64" or data[i].dtype == "float64":
        num_cols.append(i)
    else:
        pass  
    
print(f"Numeric Columns: {num_cols}")  

Numeric Columns: ['price', 'appraisal_offer', 'model_year_appraisal', 'mileage_appraisal', 'engine_appraisal', 'mpg_city_appraisal', 'mpg_highway_appraisal', 'horsepower_appraisal', 'fuel_capacity_appraisal', 'model_year', 'mileage', 'engine', 'mpg_city', 'mpg_highway', 'horsepower', 'fuel_capacity', 'days_since_offer', 'cylinders_even', 'cylinders_even_appraisal', 'cylinders_high', 'cylinders_high_appraisal', 'cylinders', 'cylinders_appraisal']


In [14]:
correlations = pd.DataFrame(data[num_cols].corr()).reset_index().rename(columns = {"index":"variable"})
correlations[["variable", "price"]].sort_values(by = "price")

,variable,price
13,mpg_highway,-0.582142
12,mpg_city,-0.477145
10,mileage,-0.379379
6,mpg_highway_appraisal,-0.277225
5,mpg_city_appraisal,-0.242899
3,mileage_appraisal,-0.190289
16,days_since_offer,0.021656
17,cylinders_even,0.028428
18,cylinders_even_appraisal,0.032865
2,model_year_appraisal,0.220073


Looks like the mileage of the car they are buying is the most negative factor in the price of the car they buy. On the otherhand, cars with higher horsepowers tend to be sold at a higher price.

### Can we drill down to see correlations between variables for appraised and purchased vehicles specifically?

In [15]:
appraised_cols = list()
purchased_cols = list()
for i in num_cols:
    if "appraisal" in i:
        appraised_cols.append(i)
    else:
        purchased_cols.append(i)

In [16]:
correlations_appraisal = pd.DataFrame(data[appraised_cols].corr()).reset_index().rename(columns = {"index":"variable"})
correlations_appraisal[["variable", "appraisal_offer"]].sort_values(by = "appraisal_offer")

,variable,appraisal_offer
2,mileage_appraisal,-0.703084
5,mpg_highway_appraisal,-0.249676
4,mpg_city_appraisal,-0.186510
8,cylinders_even_appraisal,0.046837
9,cylinders_high_appraisal,0.241081
7,fuel_capacity_appraisal,0.248726
10,cylinders_appraisal,0.280374
3,engine_appraisal,0.295929
6,horsepower_appraisal,0.449138
1,model_year_appraisal,0.701275


Looks like how new a car is is a strong predictor of the offer it will receive. As we saw with price, appraised cars with high high mileage are associated with lower appraisal offers.

In [17]:
correlations_purchased = pd.DataFrame(data[purchased_cols].corr()).reset_index().rename(columns = {"index":"variable"})
correlations_purchased[["variable", "price"]].sort_values(by = "price")

,variable,price
5,mpg_highway,-0.582142
4,mpg_city,-0.477145
2,mileage,-0.379379
8,days_since_offer,0.021656
9,cylinders_even,0.028428
1,model_year,0.440484
10,cylinders_high,0.541054
3,engine,0.577421
11,cylinders,0.584109
7,fuel_capacity,0.591591


Horsepower, fuel efficiency, and fuel capacity seem to be important to customers when they are looking at how much ther are willing to pay for a car.

### Are there differences in purchasing behavior between US Regions?

Below are the most popular makes, models, and colors across regions.

In [41]:
data[["region", "make"]].groupby("region")["make"].agg(pd.Series.mode).to_frame()

,make
region,
Midwest,Jeep
Northeast,Jeep
South,Toyota
West,Toyota


In [42]:
data[["region", "model"]].groupby("region")["model"].agg(pd.Series.mode).to_frame()

,model
region,
Midwest,Equinox
Northeast,Grand Cherokee
South,Rogue
West,Civic


In [43]:
data[["region", "color_grouped"]].groupby("region")["color_grouped"].agg(pd.Series.mode).to_frame()

,color_grouped
region,
Midwest,Black
Northeast,Black
South,Black
West,White


Let's see which region is generating the highest sales

In [50]:
data[["region", "price"]].groupby("region").sum().sort_values(ascending = False, by = "price")

,price
region,
South,1466242800
West,618706800
Midwest,284957500
Northeast,50986100


Looks like the south has generated the most in sales over the entire period of the data.

What about on average?

In [51]:
data[["region", "price"]].groupby("region").mean().sort_values(ascending=False, by="price")

,price
region,
South,21868.227714
Northeast,21751.749147
West,21714.343874
Midwest,20865.307168


Still the south!

In [54]:
data[["region", "price"]].groupby("region").median().sort_values(ascending=False, by="price")

,price
region,
Northeast,21000.0
South,21000.0
West,20200.0
Midwest,19500.0


It looks like the Northeast and South are tied for highest median, meaning South is overall the clear highest value in terms of selling cars.

### Are online appraisals higher than in-person appraisals?

In [76]:
online = data[data["online_appraisal_flag"] == True]["appraisal_offer"]
in_store = data[data["online_appraisal_flag"] == False]["appraisal_offer"]

null = "There is no difference between appraisal offers in stores and online."
alternative = "There is a difference between appraisal offers in stores and online."

alpha = 0.5

t_statistic, p_value = stats.ttest_ind(online, in_store)

if p_value <= alpha:
    decision = "Reject"
else:
    decision = "Fail to reject"

# Conclusion
if decision == "Reject":
    conclusion = "There is statistically significant evidence that the average appraisal offer is different between online and in-store appraisals."
else:
    conclusion = "There is insufficient evidence to claim a significant difference in average appraisal offer between online and in-store appraisals."

# Display results
print("T-statistic (from scipy):", t_statistic)
print("P-value (from scipy):", p_value)
print(f"Decision: {decision} the null hypothesis at alpha={alpha}.")
print("Conclusion:", conclusion)


T-statistic (from scipy): 53.7115375373089
P-value (from scipy): 0.0
Decision: Reject the null hypothesis at alpha=0.5.
Conclusion: There is statistically significant evidence that the average appraisal offer is different between online and in-store appraisals.
